In [1]:
!pip install torch torchvision torchaudio
!pip install rdkit
!pip install neat-python
!pip install numpy pandas
!pip install requests
!python -m ipykernel install --user --name cuda --display-name "Python 3 (GPU)"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.3/33.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.1 MB/s eta 0:00:00
Installed kernelspec cuda in /root/.local/share/jupyter/kernels/cuda


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, QED, Draw
from rdkit.Chem import DataStructs
import logging
from typing import List, Dict, Optional, Tuple
import warnings
import requests
import gc
import random
from pathlib import Path
from torch.cuda import empty_cache

warnings.filterwarnings('ignore')

class HypergraphLayer(nn.Module):
    """Memory-optimized hypergraph layer for molecular representation"""
    def __init__(self, in_features, out_features, num_edges=32):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_edges = num_edges

        # Hyperedge projections with memory optimization
        self.edge_projs = nn.Parameter(
            torch.empty(num_edges, in_features, out_features, dtype=torch.float32)
        )
        self.attention = nn.Parameter(
            torch.empty(num_edges, out_features, dtype=torch.float32)
        )

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.edge_projs)
        nn.init.xavier_uniform_(self.attention)

    def forward(self, x):
        chunk_size = min(32, x.size(0))
        outputs = []

        for i in range(0, x.size(0), chunk_size):
            chunk = x[i:i + chunk_size]
            multi_head = torch.stack([
                torch.matmul(chunk, proj) for proj in self.edge_projs
            ])

            attn_weights = F.softmax(
                torch.matmul(multi_head, self.attention.unsqueeze(-1)),
                dim=0
            )

            out = torch.sum(multi_head * attn_weights, dim=0)
            outputs.append(out)

            del multi_head, attn_weights

        return torch.cat(outputs, dim=0)

class MemoryEfficientDataset(Dataset):
    """Memory-efficient dataset implementation with batch processing"""
    def __init__(self, csv_file: str, fp_size: int = 2048, cache_size: int = 10000):
        self.fp_size = fp_size
        self.cache_size = min(cache_size, sum(1 for _ in open(csv_file)))
        self.features = []
        self.smiles = []

        # Load initial cache
        print("Loading initial data cache...")
        df = pd.read_csv(csv_file, nrows=self.cache_size)

        # Process molecules in parallel using multiple threads
        from concurrent.futures import ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = []
            for smiles in df['smiles']:
                futures.append(executor.submit(self._process_molecule, smiles))

            for future in futures:
                result = future.result()
                if result is not None:
                    fp, smiles = result
                    self.features.append(fp)
                    self.smiles.append(smiles)

        # Convert to numpy array for faster access
        self.features = np.array(self.features, dtype=np.float32)
        print(f"Loaded {len(self.features)} molecules into cache")

    def _process_molecule(self, smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                fp = AllChem.GetMorganFingerprintAsBitVect(
                    mol, 2, nBits=self.fp_size)
                arr = np.zeros((1,), dtype=np.float32)
                DataStructs.ConvertToNumpyArray(fp, arr)
                return arr, smiles
        except:
            pass
        return None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.features[idx])

    def count_valid_molecules(self):
        chunk_size = 1000
        total = 0
        for chunk in pd.read_csv(self.csv_file, chunksize=chunk_size):
            for smiles in chunk['smiles']:
                try:
                    mol = Chem.MolFromSmiles(smiles)
                    if mol is not None:
                        total += 1
                except:
                    continue
        self.total_size = total

    def process_batch(self, start_idx: int):
        self.features = []
        self.smiles = []

        chunk_size = 1000
        processed = 0
        current_idx = 0

        for chunk in pd.read_csv(self.csv_file, chunksize=chunk_size):
            for smiles in chunk['smiles']:
                if current_idx >= start_idx and processed < self.batch_size:
                    try:
                        mol = Chem.MolFromSmiles(smiles)
                        if mol is not None:
                            fp = AllChem.GetMorganFingerprintAsBitVect(
                                mol, 2, nBits=self.fp_size)
                            arr = np.zeros((1,), dtype=np.float32)
                            DataStructs.ConvertToNumpyArray(fp, arr)

                            self.features.append(arr)
                            self.smiles.append(smiles)
                            processed += 1
                    except Exception as e:
                        continue
                current_idx += 1

                if processed >= self.batch_size:
                    break
            if processed >= self.batch_size:
                break

    # def __len__(self):
    #     return self.total_size

    # def __getitem__(self, idx):
    #     batch_idx = idx // self.batch_size
    #     local_idx = idx % self.batch_size

    #     if local_idx == 0:
    #         self.process_batch(batch_idx * self.batch_size)

    #     return torch.FloatTensor(self.features[local_idx])

def download_zinc_dataset(url: str, output_path: str) -> bool:
    """Download the ZINC dataset"""
    try:
        print(f"Downloading ZINC dataset from {url}...")
        response = requests.get(url)
        response.raise_for_status()

        with open(output_path, 'wb') as f:
            f.write(response.content)

        print(f"Dataset downloaded to {output_path}")
        return True
    except Exception as e:
        logging.error(f"Download failed: {str(e)}")
        return False

class ElectroceuticalVAE(nn.Module):
    """Memory-optimized VAE with electroceutical-specific architecture"""
    def __init__(self, input_dim=2048, hidden_dim=512, latent_dim=128):
        super().__init__()

        self.encoder = nn.Sequential(
            HypergraphLayer(input_dim, hidden_dim, num_edges=16),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.2)
        )

        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_var = nn.Linear(hidden_dim, latent_dim)

        # Remove sigmoid from decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, input_dim)  # No sigmoid here
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_var(h)

    def decode(self, z):
        return self.decoder(z)

    def reparameterize(self, mu, log_var):
        if self.training:
            std = torch.exp(0.5 * log_var)
            eps = torch.randn_like(std)
            return mu + eps * std
        return mu

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        return self.decode(z), mu, log_var

class ElectroceuticalProfile: # for cancer treatment
    """Enhanced profile for electroceutical drug evaluation"""
    def __init__(self):
        self.target_profiles = {
            'ion_channels': {
                'potassium': {
                    'activation_threshold': 0.8,
                    'voltage_range': (-90, -70),
                    'specificity': 0.7,
                    'structural_features': {
                        'required_groups': [
                            'C(=O)N',  # For membrane interaction
                            'CN',      # For voltage modulation
                            '[N+]',    # For charge sensitivity
                            'c1ccccc1' # For cancer cell penetration
                        ],
                        'favorable_props': {
                            'mw_range': (300, 750),  # Extended for cancer drugs
                            'logp_range': (1, 5)     # Modified for cancer cell penetration
                        }
                    }
                },
                'cancer_ion_regulation': {  # New cancer-specific target
                    'modulation_threshold': 0.75,
                    'voltage_range': (-30, 10),
                    'specificity': 0.85,
                    'cancer_specific': True,
                    'required_features': [
                        'C(=O)N1CCN(C)CC1',  # Kinase binding
                        'c1ccc2ncncc2c1',    # Cancer cell targeting
                        'CC(=O)NC1=CC=C(F)C=C1'  # Anti-cancer motif
                    ]
                }
            },
            'membrane_potential': {
                'cancer_depolarization': {  # Cancer-specific depolarization
                    'threshold': -20,
                    'duration': 50,
                    'cancer_specific': True,
                    'required_features': [
                        '[O-]', '[N+]',
                        'C(=O)N1CCN(C)CC1',  # Cancer targeting
                        'c1ccc2ncncc2c1'     # Privileged scaffold
                    ]
                }
            },
            'bioelectric_signaling': {
                'cancer_voltage_gated': {
                    'activation_voltage': -50,
                    'inactivation_voltage': -30,
                    'cancer_specific': True,
                    'required_features': [
                        '[N+]', '[O-]',
                        'c1ccc(-n2c(=O)[nH]c3ccccc32)cc1'  # Kinase binding
                    ]
                }
            }
        }

    def evaluate_molecule(self, mol: Chem.Mol, target_type: str,
                        target_name: str) -> Dict:
        try:
            if target_type not in self.target_profiles:
                raise ValueError(f"Unknown target type: {target_type}")

            if target_name not in self.target_profiles[target_type]:
                raise ValueError(f"Unknown target name: {target_name}")

            profile = self.target_profiles[target_type][target_name]

            bioelectric_score = self._calculate_bioelectric_score(mol, profile)
            specificity_score = self._calculate_specificity_score(mol, profile)
            activity_score = self._calculate_activity_score(mol, profile)
            safety_score = self._calculate_safety_score(mol)
            properties = self._calculate_molecular_properties(mol)

            total_score = np.mean([
                bioelectric_score,
                specificity_score,
                activity_score,
                safety_score
            ])

            return {
                'total_score': total_score,
                'bioelectric_score': bioelectric_score,
                'specificity_score': specificity_score,
                'activity_score': activity_score,
                'safety_score': safety_score,
                'properties': properties
            }

        except Exception as e:
            logging.error(f"Molecule evaluation failed: {str(e)}")
            return None

    def _calculate_bioelectric_score(self, mol: Chem.Mol, profile: Dict) -> float:
        try:
            # Original bioelectric patterns
            patterns = [
                '[N+]', '[O-]',
                'S(=O)(=O)[O-]',
                'P(=O)([O-])[O-]',
                'C(=O)[O-]'
            ]

            # Cancer-specific patterns
            cancer_patterns = [
                'c1ccc(-n2c(=O)[nH]c3ccccc32)cc1',  # Kinase binding
                'CC(=O)N1CCN(C)CC1',                 # Cancer penetration
                'C(=O)NC1=CC=C(F)C=C1',             # Anti-cancer motif
            ]

            score = 0
            # Score regular patterns
            for pattern in patterns:
                patt = Chem.MolFromSmarts(pattern)
                if patt and mol.HasSubstructMatch(patt):
                    score += 0.1

            # Additional score for cancer-specific molecules
            if profile.get('cancer_specific'):
                for pattern in cancer_patterns:
                    patt = Chem.MolFromSmarts(pattern)
                    if patt and mol.HasSubstructMatch(patt):
                        score += 0.15

            # Consider cancer-specific membrane properties
            if 'voltage_range' in profile:
                tpsa = Descriptors.TPSA(mol)
                logp = Descriptors.MolLogP(mol)

                # TPSA range optimal for cancer cell penetration
                if 70 <= tpsa <= 180:
                    score += 0.1
                # LogP range for cancer cell membrane interaction
                if 2 <= logp <= 5:
                    score += 0.1

            return min(1.0, score)

        except Exception as e:
            logging.warning(f"Bioelectric score calculation failed: {str(e)}")
            return 0.0

    def _calculate_specificity_score(self, mol: Chem.Mol, profile: Dict) -> float:
        try:
            if 'specificity' not in profile:
                return 1.0

            target_specificity = profile['specificity']

            mw = Descriptors.MolWt(mol)
            logp = Descriptors.MolLogP(mol)
            rotatable = Descriptors.NumRotatableBonds(mol)

            mw_score = 1.0 if 250 <= mw <= 500 else max(0, 1 - abs(mw - 375) / 375)
            logp_score = 1.0 if 1 <= logp <= 4 else max(0, 1 - abs(logp - 2.5) / 2.5)
            flex_score = 1.0 if 2 <= rotatable <= 8 else max(0, 1 - abs(rotatable - 5) / 5)

            return (mw_score + logp_score + flex_score) * target_specificity / 3

        except Exception as e:
            logging.warning(f"Specificity calculation failed: {str(e)}")
            return 0.0

    def _calculate_cancer_specificity(self, mol: Chem.Mol, profile: Dict) -> float:
        """Calculate cancer-specific targeting score"""
        try:
            if not profile.get('cancer_specific'):
                return self._calculate_specificity_score(mol, profile)

            # Cancer-specific structural features
            cancer_features = [
                'c1ccc2ncncc2c1',    # Kinase binding scaffold
                'C(=O)NC1=CC=CC=C1',  # Protein binding motif
                'n1ccccc1C(=O)',      # ATP binding region
                'CN1CCN(C)CC1'        # Cancer cell penetration
            ]

            score = 0.0
            for feature in cancer_features:
                patt = Chem.MolFromSmarts(feature)
                if patt and mol.HasSubstructMatch(patt):
                    score += 0.25

            # Cancer-relevant properties
            mw = Descriptors.MolWt(mol)
            logp = Descriptors.MolLogP(mol)
            tpsa = Descriptors.TPSA(mol)

            # Scoring based on optimal ranges for cancer drugs
            if 300 <= mw <= 750: score += 0.1
            if 2 <= logp <= 5: score += 0.1
            if 70 <= tpsa <= 180: score += 0.1

            return min(1.0, score)

        except Exception as e:
            logging.warning(f"Cancer specificity calculation failed: {str(e)}")
            return 0.0

    def _calculate_activity_score(self, mol: Chem.Mol, profile: Dict) -> float:
        """Calculate activity score with enhanced cancer targeting"""
        try:
            if profile.get('cancer_specific'):
                return self._calculate_cancer_activity(mol, profile)

            # Original activity calculation for non-cancer targets
            return super()._calculate_activity_score(mol, profile)

        except Exception as e:
            logging.warning(f"Activity score calculation failed: {str(e)}")
            return 0.0

    def _calculate_cancer_activity(self, mol: Chem.Mol, profile: Dict) -> float:
        """Calculate cancer-specific activity score"""
        try:
            # Cancer-relevant patterns
            kinase_patterns = [
                'c1ccc(-n2c(=O)[nH]c3ccccc32)cc1',
                'CN1CCN(C(=O)c2cccc(F)c2)CC1'
            ]

            apoptosis_patterns = [
                'c1ccc2c(c1)nc(N)n2',
                'C(=O)NC1=CC=CC=C1'
            ]

            metabolic_patterns = [
                'C(=O)O',     # Glycolysis interaction
                'CC(=O)C',    # Ketone groups
                'CN(C)C=O'    # Energy metabolism
            ]

            # Calculate specific scores
            kinase_score = sum(1 for p in kinase_patterns
                             if mol.HasSubstructMatch(Chem.MolFromSmarts(p))) / len(kinase_patterns)

            apoptosis_score = sum(1 for p in apoptosis_patterns
                                if mol.HasSubstructMatch(Chem.MolFromSmarts(p))) / len(apoptosis_patterns)

            metabolic_score = sum(1 for p in metabolic_patterns
                                if mol.HasSubstructMatch(Chem.MolFromSmarts(p))) / len(metabolic_patterns)

            # Weighted combination
            return (0.4 * kinase_score + 0.3 * apoptosis_score + 0.3 * metabolic_score)

        except Exception as e:
            logging.warning(f"Cancer activity calculation failed: {str(e)}")
            return 0.0

    def _calculate_molecular_properties(self, mol: Chem.Mol) -> Dict:
        """Calculate basic molecular properties"""
        try:
            return {
                'MW': Descriptors.MolWt(mol),
                'LogP': Descriptors.MolLogP(mol),
                'TPSA': Descriptors.TPSA(mol),
                'HBA': Descriptors.NumHAcceptors(mol),
                'HBD': Descriptors.NumHDonors(mol),
                'RotatableBonds': Descriptors.NumRotatableBonds(mol),
                'QED': QED.default(mol)
            }
        except Exception as e:
            logging.error(f"Property calculation failed: {str(e)}")
            return {}

    def _calculate_safety_score(self, mol: Chem.Mol) -> float:
        """Calculate safety profile with fixed patterns"""
        try:
            toxic_patterns = [
                '*=N(~[OH])~[OH]',  # N-oxide
                'C(=O)Cl',          # Acid chloride
                'N(~O)(~O)',        # Nitro group
                '[N+](=O)[O-]',     # Nitro group
                'C1=CC=C2C(=C1)C(=O)C=CC2=O'  # Quinone
            ]

            score = 1.0
            for pattern in toxic_patterns:
                patt = Chem.MolFromSmarts(pattern)
                if patt and mol.HasSubstructMatch(patt):
                    score -= 0.2

            return max(0.0, score)

        except Exception as e:
            logging.warning(f"Safety score calculation failed: {str(e)}")
            return 0.0

class OptimizedElectroceuticalGenerator:
    """Memory-optimized generator for electroceutical drug design"""
    def __init__(self, fp_size: int = 2048):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.fp_size = fp_size

        # Initialize models
        self.model = ElectroceuticalVAE(
            input_dim=fp_size,
            hidden_dim=512,
            latent_dim=128
        ).to(self.device)

        # Use mixed precision training
        self.scaler = torch.cuda.amp.GradScaler()

        # Optimizer with memory optimization
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=1e-3,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=0
        )

        self.electro_profile = ElectroceuticalProfile()
        self.dataset = None

    def train(self, dataset_path: str, epochs: int = 50, batch_size: int = 32):
        """Optimized training loop with BCEWithLogitsLoss"""
        try:
            self.dataset = MemoryEfficientDataset(dataset_path, self.fp_size)

            train_loader = DataLoader(
                self.dataset,
                batch_size=batch_size,
                shuffle=True,
                pin_memory=True,
                num_workers=4,
                prefetch_factor=2
            )

            torch.backends.cudnn.benchmark = True
            self.model.train()

            # Use BCEWithLogitsLoss for stable autocast
            bce_loss = nn.BCEWithLogitsLoss(reduction='sum')

            for epoch in range(epochs):
                total_loss = 0
                for batch_idx, data in enumerate(train_loader):
                    data = data.to(self.device, non_blocking=True)
                    self.optimizer.zero_grad(set_to_none=True)

                    with torch.cuda.amp.autocast():
                        recon_batch, mu, log_var = self.model(data)
                        # Use BCEWithLogitsLoss instead of sigmoid + BCE
                        reconstruction_loss = bce_loss(recon_batch, data)
                        kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
                        bio_loss = self._bioelectric_loss(torch.sigmoid(recon_batch))  # Apply sigmoid here

                        loss = reconstruction_loss + kl_loss + 0.1 * bio_loss

                    self.scaler.scale(loss).backward()
                    self.scaler.step(self.optimizer)
                    self.scaler.update()

                    total_loss += loss.item()

                    if batch_idx % 50 == 0:
                        torch.cuda.empty_cache()

                avg_loss = total_loss / len(train_loader.dataset)
                if (epoch + 1) % 5 == 0:
                    print(f'Epoch: {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

        except Exception as e:
            logging.error(f"Training failed: {str(e)}")
            raise e  # Raise the actual exception

    def _calculate_cancer_activity(self, mol: Chem.Mol, profile: Dict) -> float:
      """Calculate cancer-specific activity score with enhanced targeting"""
      try:
          # Enhanced cancer-relevant patterns
          kinase_patterns = [
              'c1ccc(-n2c(=O)[nH]c3ccccc32)cc1',  # Kinase binding
              'CN1CCN(C(=O)c2cccc(F)c2)CC1',      # Cell penetration
              'c1ccc2[nH]c(=O)nc2c1',             # ATP binding
              'CC(=O)NC1=CC=C(F)C=C1',            # Targeting moiety
              'c1cnc2[nH]ccc2c1'                   # Hinge binding
          ]

          apoptosis_patterns = [
              'c1ccc2c(c1)nc(N)n2',          # DNA intercalation
              'C(=O)NC1=CC=CC=C1',           # Apoptosis trigger
              'CN1C(=O)CN=C(C2=CC=CC=C2)C1', # Death receptor binding
              'CC(=O)NC(=O)C1=CC=CC=C1'      # Caspase activation
          ]

          metabolic_patterns = [
              'C(=O)O',         # Glycolysis interaction
              'CC(=O)C',        # Ketone groups
              'CN(C)C=O',       # Energy metabolism
              'C(=O)CC(=O)',    # TCA cycle interaction
              'CC(=O)OC'        # Mitochondrial targeting
          ]

          bioelectric_patterns = [
              '[N+]',           # Ion channel interaction
              'S(=O)(=O)[O-]',  # Membrane potential
              'P(=O)([O-])[O-]',# ATP synthesis
              'C(=O)[O-]',      # Proton gradient
              'CN1CCN(C)CC1'    # Voltage-gated channels
          ]

          # Calculate specific scores with weights
          def calculate_pattern_score(patterns):
              matches = 0
              total_patterns = len(patterns)
              for pattern in patterns:
                  patt = Chem.MolFromSmarts(pattern)
                  if patt and mol.HasSubstructMatch(patt):
                      matches += 1
              return matches / total_patterns if total_patterns > 0 else 0

          kinase_score = calculate_pattern_score(kinase_patterns)
          apoptosis_score = calculate_pattern_score(apoptosis_patterns)
          metabolic_score = calculate_pattern_score(metabolic_patterns)
          bioelectric_score = calculate_pattern_score(bioelectric_patterns)

          # Calculate physicochemical properties relevant for cancer
          mw = Descriptors.MolWt(mol)
          logp = Descriptors.MolLogP(mol)
          tpsa = Descriptors.TPSA(mol)
          rotatable_bonds = Descriptors.NumRotatableBonds(mol)

          # Score physicochemical properties
          mw_score = 1.0 if 300 <= mw <= 750 else max(0, 1 - abs(mw - 525) / 525)
          logp_score = 1.0 if 2 <= logp <= 5 else max(0, 1 - abs(logp - 3.5) / 3.5)
          tpsa_score = 1.0 if 70 <= tpsa <= 180 else max(0, 1 - abs(tpsa - 125) / 125)
          flex_score = 1.0 if 3 <= rotatable_bonds <= 10 else max(0, 1 - abs(rotatable_bonds - 6.5) / 6.5)

          # Weighted combination of all scores
          mechanism_score = (
              0.35 * kinase_score +      # Primary targeting
              0.25 * apoptosis_score +   # Cell death
              0.20 * metabolic_score +   # Metabolic disruption
              0.20 * bioelectric_score   # Ion channel modulation
          )

          property_score = (
              0.3 * mw_score +
              0.3 * logp_score +
              0.2 * tpsa_score +
              0.2 * flex_score
          )

          # Final score combining mechanism and properties
          return 0.7 * mechanism_score + 0.3 * property_score

      except Exception as e:
          logging.warning(f"Cancer activity calculation failed: {str(e)}")
          return 0.0

    # Update generate_molecules to handle logits
    def _add_cancer_targeting_group(self, mol: Chem.RWMol) -> Optional[Chem.RWMol]:
      """Add cancer-specific targeting groups with enhanced bioelectric properties"""
      cancer_groups = [
          # Kinase targeting groups
          'c1ccc(-n2c(=O)[nH]c3ccccc32)cc1',    # Kinase binding
          'CN1CCN(C(=O)c2cccc(F)c2)CC1',        # Cell penetration

          # Ion channel modulators
          'CC(=O)NC1=CC=C(F)C=C1[N+]',          # Voltage-gated channel
          'c1cnc2[nH]ccc2c1[O-]',               # ATP binding + ion modulation

          # Membrane potential modifiers
          'CN1CCN(CC(=O)O)CC1',                 # Proton pump interaction
          'C[N+]1=CC=CC(S(=O)(=O)[O-])=C1',    # Membrane voltage modifier

          # Dual-action groups
          'CC(=O)N1CCN(CC2=CC=C(F)C=C2)CC1',    # Channel blocker + kinase
          'C[N+]1=CC=C(NC(=O)C2=CC=CC=C2)C=C1'  # Ion transport + apoptosis
      ]

      try:
          if not isinstance(mol, Chem.RWMol):
              mol = Chem.RWMol(mol)

          # Choose multiple groups to add
          n_groups = random.randint(1, 3)
          selected_groups = random.sample(cancer_groups, n_groups)

          modified_mol = mol
          for group_smiles in selected_groups:
              try:
                  group = Chem.MolFromSmiles(group_smiles)
                  if group is None:
                      continue

                  # Find suitable attachment points
                  mol_atoms = [(i, atom) for i, atom in enumerate(modified_mol.GetAtoms())
                              if atom.GetTotalValence() < self._get_max_valence(atom.GetAtomicNum())]

                  group_atoms = [(i, atom) for i, atom in enumerate(group.GetAtoms())
                                if atom.GetTotalValence() < self._get_max_valence(atom.GetAtomicNum())]

                  if not mol_atoms or not group_atoms:
                      continue

                  # Choose attachment points
                  mol_idx, _ = random.choice(mol_atoms)
                  group_idx, _ = random.choice(group_atoms)

                  # Combine molecules
                  combo = Chem.CombineMols(modified_mol, group)
                  editable = Chem.RWMol(combo)

                  # Add bond
                  editable.AddBond(mol_idx,
                                group_idx + modified_mol.GetNumAtoms(),
                                Chem.BondType.SINGLE)

                  # Verify and update
                  try:
                      Chem.SanitizeMol(editable)
                      modified_mol = editable
                  except:
                      continue

              except Exception as e:
                  logging.warning(f"Failed to add group {group_smiles}: {str(e)}")
                  continue

          return modified_mol

      except Exception as e:
          logging.error(f"Cancer targeting group addition failed: {str(e)}")
          return mol

    def _optimize_bioelectric_properties(self, mol: Chem.RWMol) -> Optional[Chem.RWMol]:
      """Optimize molecule's bioelectric properties for cancer treatment"""
      try:
          # Define modification patterns
          modifications = [
              # Ion channel modulators
              ('[N+]', 'C[N+](C)(C)C'),
              ('[O-]', 'CC(=O)[O-]'),
              ('S(=O)(=O)O', 'S(=O)(=O)[O-]'),

              # Membrane potential modifiers
              ('CN', 'CN1CCN(C)CC1'),
              ('CO', 'COC(=O)C[N+](C)(C)C'),

              # Proton pump interactors
              ('C(=O)O', 'C(=O)O[H]'),
              ('CN(C)C', 'CN(C)CC(=O)[O-]')
          ]

          modified_mol = Chem.RWMol(mol)

          # Apply random modifications
          n_mods = random.randint(1, 3)
          for _ in range(n_mods):
              pattern, replacement = random.choice(modifications)

              # Find matches for the pattern
              patt = Chem.MolFromSmarts(pattern)
              if patt and modified_mol.HasSubstructMatch(patt):
                  matches = modified_mol.GetSubstructMatches(patt)
                  if matches:
                      match = random.choice(matches)

                      # Replace with bioelectric group
                      repl = Chem.MolFromSmiles(replacement)
                      if repl:
                          try:
                              rw_mol = Chem.RWMol(modified_mol)
                              rw_mol = Chem.ReplaceSubstructs(rw_mol, patt, repl)[0]
                              Chem.SanitizeMol(rw_mol)
                              modified_mol = rw_mol
                          except:
                              continue

          return modified_mol

      except Exception as e:
          logging.error(f"Bioelectric property optimization failed: {str(e)}")
          return mol

    def generate_molecules(self, n_samples: int = 10,
                      target_type: str = 'ion_channels',
                      target_name: str = 'cancer_ion_regulation',
                      min_score: float = 0.6) -> List[Tuple[Optional[Chem.Mol], Dict]]:
        """Generate molecules with detailed logging and cancer-specific optimization"""
        self.model.eval()
        generated_mols = []
        batch_size = 5
        attempts = 0
        max_attempts = n_samples * 100

        print("\nStarting molecule generation...")
        print(f"Target: {target_type}/{target_name}")
        print(f"Minimum score threshold: {min_score}")

        try:
            with torch.no_grad():
                while len(generated_mols) < n_samples and attempts < max_attempts:
                    attempts += 1
                    print(f"\nAttempt {attempts}/{max_attempts}")

                    # Generate latent vectors with cancer-specific bias
                    z = torch.randn(batch_size, self.model.fc_mu.out_features,
                                  device=self.device, dtype=torch.float32)

                    # Add bias towards cancer-relevant regions
                    if target_name == 'cancer_ion_regulation':
                        z += 0.2 * torch.sign(z)  # Enhance extreme values

                    # Generate molecules
                    decoded = self.model.decode(z)
                    decoded = torch.sigmoid(decoded).cpu().numpy()

                    for fp_idx, fp in enumerate(decoded):
                        print(f"\nProcessing fingerprint {fp_idx + 1}/{len(decoded)}")

                        # Convert fingerprint to molecule
                        mol = self._fingerprint_to_mol(fp)

                        if mol is not None:
                            try:
                                # Apply cancer-specific modifications if applicable
                                if target_name == 'cancer_ion_regulation':
                                    print("Applying cancer-specific modifications...")
                                    try:
                                        modified_mol = Chem.RWMol(mol)

                                        # Add targeting groups
                                        modified_mol = self._add_cancer_targeting_group(modified_mol)
                                        if modified_mol is None:
                                            print("Failed to add cancer targeting groups")
                                            continue

                                        # Optimize bioelectric properties
                                        modified_mol = self._optimize_bioelectric_properties(modified_mol)
                                        if modified_mol is None:
                                            print("Failed to optimize bioelectric properties")
                                            continue

                                        mol = modified_mol
                                    except Exception as e:
                                        print(f"Cancer modification failed: {str(e)}")
                                        continue

                                # Check basic validity
                                Chem.SanitizeMol(mol)
                                smiles = Chem.MolToSmiles(mol)
                                print(f"Valid molecule generated: {smiles}")

                                # Check atom valences
                                valence_valid = True
                                for atom in mol.GetAtoms():
                                    atom.UpdatePropertyCache(strict=False)
                                    if atom.GetTotalValence() > self._get_max_valence(atom.GetAtomicNum()):
                                        print(f"Invalid valence for atom {atom.GetSymbol()} at position {atom.GetIdx()}")
                                        print(f"Total valence: {atom.GetTotalValence()}")
                                        print(f"Max allowed: {self._get_max_valence(atom.GetAtomicNum())}")
                                        valence_valid = False
                                        break

                                if not valence_valid:
                                    print("Skipping molecule due to valence issues")
                                    continue

                                # Additional checks for cancer molecules
                                if target_name == 'cancer_ion_regulation':
                                    # Check for minimum requirements
                                    required_features = [
                                        '[N+]',         # Ion channel interaction
                                        'c1ccccc1',     # Aromatic ring
                                        'C(=O)N',       # Amide group
                                    ]
                                    features_present = sum(1 for feat in required_features
                                                         if mol.HasSubstructMatch(Chem.MolFromSmarts(feat)))

                                    if features_present < 2:  # At least 2 features required
                                        print("Molecule lacks required cancer-targeting features")
                                        continue

                                # Evaluate molecule
                                eval_results = self.electro_profile.evaluate_molecule(
                                    mol, target_type, target_name
                                )

                                if eval_results:
                                    print("\nEvaluation results:")
                                    print(f"Total score: {eval_results['total_score']:.3f}")
                                    print(f"Bioelectric score: {eval_results['bioelectric_score']:.3f}")
                                    print(f"Specificity score: {eval_results['specificity_score']:.3f}")
                                    print(f"Activity score: {eval_results['activity_score']:.3f}")
                                    print(f"Safety score: {eval_results['safety_score']:.3f}")

                                    # Log molecular properties
                                    print("\nMolecular properties:")
                                    for prop, value in eval_results['properties'].items():
                                        print(f"{prop}: {value:.2f}")

                                    if eval_results['total_score'] >= min_score:
                                        # Additional cancer-specific property checks
                                        if target_name == 'cancer_ion_regulation':
                                            mw = eval_results['properties']['MW']
                                            logp = eval_results['properties']['LogP']
                                            tpsa = eval_results['properties']['TPSA']

                                            if not (300 <= mw <= 750 and
                                                   1 <= logp <= 5 and
                                                   70 <= tpsa <= 180):
                                                print("\nMolecule rejected - outside cancer-specific property ranges")
                                                continue

                                        print("\nMolecule accepted ✓")
                                        generated_mols.append((mol, eval_results))
                                    else:
                                        print(f"\nMolecule rejected - score {eval_results['total_score']:.3f} below threshold {min_score}")
                                else:
                                    print("Evaluation failed")
                            except Exception as e:
                                print(f"Error processing molecule: {str(e)}")
                                continue
                        else:
                            print("Failed to convert fingerprint to molecule")

                        if len(generated_mols) >= n_samples:
                            break

                    # Memory cleanup
                    del z, decoded
                    torch.cuda.empty_cache()
                    gc.collect()

                print(f"\nGeneration complete. Generated {len(generated_mols)} valid molecules")
                print(f"Total attempts: {attempts}")

                # Final summary
                print("\nGenerated molecules summary:")
                for i, (mol, results) in enumerate(generated_mols, 1):
                    print(f"\nMolecule {i}:")
                    print(f"SMILES: {Chem.MolToSmiles(mol)}")
                    print(f"Total Score: {results['total_score']:.3f}")
                    print("-" * 40)

                return generated_mols[:n_samples]

        except Exception as e:
            logging.error(f"Generation failed: {str(e)}")
            raise e

    def _check_molecule_validity(self, mol: Chem.Mol) -> Tuple[bool, str]:
        """Check molecule validity and return reason if invalid"""
        try:
            if mol is None:
                return False, "Molecule is None"

            # Check basic validity
            try:
                Chem.SanitizeMol(mol)
            except Exception as e:
                return False, f"Sanitization failed: {str(e)}"

            # Check valences
            for atom in mol.GetAtoms():
                atom.UpdatePropertyCache(strict=False)
                if atom.GetTotalValence() > self._get_max_valence(atom.GetAtomicNum()):
                    return False, f"Invalid valence for {atom.GetSymbol()} atom at position {atom.GetIdx()}"

            # Check for disconnected fragments
            if len(Chem.GetMolFrags(mol)) > 1:
                return False, "Molecule contains disconnected fragments"

            return True, "Valid molecule"

        except Exception as e:
          return False, f"Validation check failed: {str(e)}"

    def _bioelectric_loss(self, decoded: torch.Tensor) -> torch.Tensor:
        """Calculate bioelectric regularization loss"""
        charge_patterns = torch.sum(decoded[:, ::4], dim=1)
        membrane_patterns = torch.sum(decoded[:, 1::4], dim=1)

        charge_loss = 1.0 - torch.mean(charge_patterns)
        membrane_loss = 1.0 - torch.mean(membrane_patterns)

        return charge_loss + membrane_loss

    def _loss_function(self, recon_x: torch.Tensor, x: torch.Tensor,
                      mu: torch.Tensor, log_var: torch.Tensor) -> torch.Tensor:
        BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
        KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        return BCE + KLD

    def _add_charged_group(self, mol: Chem.RWMol) -> Optional[Chem.RWMol]:
        groups = [
            '[N+](C)(C)(C)C',
            'C(=O)[O-]',
            'S(=O)(=O)[O-]',
            'P(=O)([O-])[O-]'
        ]
        return self._add_group(mol, groups)

    def _add_membrane_interactive_group(self, mol: Chem.RWMol) -> Optional[Chem.RWMol]:
        groups = [
            'CCCCC',
            'c1ccccc1',
            'CCCCN',
            'CCCCO'
        ]
        return self._add_group(mol, groups)

    def _add_voltage_sensitive_group(self, mol: Chem.RWMol) -> Optional[Chem.RWMol]:
        groups = [
            'c1ccc(N)cc1',
            'c1ccc(O)cc1',
            'C=CC(=O)N',
            'C1=CN=CN1'
        ]
        return self._add_group(mol, groups)

    def _add_group(self, mol: Chem.RWMol, groups: List[str]) -> Optional[Chem.RWMol]:
        """Add functional group with proper valence handling"""
        try:
            group_smiles = random.choice(groups)
            group = Chem.MolFromSmiles(group_smiles)
            if group is None:
                return mol

            # Make editable copies
            mol_edit = Chem.RWMol(mol)
            group_edit = Chem.RWMol(group)

            # Calculate explicit valence for all atoms
            for atom in mol_edit.GetAtoms():
                atom.UpdatePropertyCache(strict=False)
            for atom in group_edit.GetAtoms():
                atom.UpdatePropertyCache(strict=False)

            # Find suitable attachment points
            mol_atoms = [(i, atom) for i, atom in enumerate(mol_edit.GetAtoms())
                        if atom.GetTotalValence() < atom.GetImplicitValence()]
            group_atoms = [(i, atom) for i, atom in enumerate(group_edit.GetAtoms())
                          if atom.GetTotalValence() < atom.GetImplicitValence()]

            if not mol_atoms or not group_atoms:
                return mol

            # Choose atoms that can form bonds
            mol_idx, mol_atom = random.choice(mol_atoms)
            group_idx, group_atom = random.choice(group_atoms)

            # Check valence and form bond
            try:
                combo = Chem.CombineMols(mol_edit, group_edit)
                editable = Chem.RWMol(combo)

                # Update property cache before adding bond
                for atom in editable.GetAtoms():
                    atom.UpdatePropertyCache(strict=False)

                editable.AddBond(mol_idx,
                              group_idx + mol_edit.GetNumAtoms(),
                              Chem.BondType.SINGLE)

                # Sanitize with error handling
                try:
                    Chem.SanitizeMol(editable)
                    return editable
                except:
                    return mol
            except:
                return mol

        except Exception as e:
            logging.error(f"Failed to add group: {str(e)}")
            return mol

    def _modify_molecule(self, mol: Chem.Mol) -> Optional[Chem.Mol]:
        """Enhanced molecule modification with cancer targeting"""
        try:
            editable_mol = Chem.RWMol(mol)

            # Update property cache
            for atom in editable_mol.GetAtoms():
                atom.UpdatePropertyCache(strict=False)

            # Include cancer-specific modifications
            modifications = [
                self._add_charged_group,
                self._add_membrane_interactive_group,
                self._modify_ionizable_groups,
                self._add_voltage_sensitive_group,
                self._add_cancer_targeting_group  # New cancer-specific modification
            ]

            n_modifications = np.random.randint(1, 3)
            for _ in range(n_modifications):
                modification = random.choice(modifications)
                new_mol = modification(editable_mol)
                if new_mol is not None:
                    editable_mol = new_mol

            try:
                final_mol = editable_mol.GetMol()
                Chem.SanitizeMol(final_mol)
                return final_mol
            except:
                return mol

        except Exception as e:
            logging.error(f"Molecule modification failed: {str(e)}")
            return None

    def _add_group_with_valence_check(self, mol: Chem.RWMol, groups: List[str]) -> Optional[Chem.RWMol]:
      """Add functional group with strict valence checking"""
      try:
          group_smiles = random.choice(groups)
          group = Chem.MolFromSmiles(group_smiles)
          if group is None:
              return mol

          # Make editable copies
          mol_edit = Chem.RWMol(mol)
          group_edit = Chem.RWMol(group)

          # Update property cache for valence checking
          for atom in mol_edit.GetAtoms():
              atom.UpdatePropertyCache(strict=False)
          for atom in group_edit.GetAtoms():
              atom.UpdatePropertyCache(strict=False)

          # Find atoms that can accept new bonds
          def get_valid_atoms(mol):
              valid_atoms = []
              for i, atom in enumerate(mol.GetAtoms()):
                  # Calculate current and maximum valence
                  curr_valence = atom.GetTotalValence()
                  max_valence = self._get_max_valence(atom.GetAtomicNum())

                  # Check if atom can accept more bonds
                  if curr_valence < max_valence:
                      valid_atoms.append((i, atom))
              return valid_atoms

          mol_atoms = get_valid_atoms(mol_edit)
          group_atoms = get_valid_atoms(group_edit)

          if not mol_atoms or not group_atoms:
              return mol

          # Try to form valid bonds
          for mol_idx, mol_atom in mol_atoms:
              for group_idx, group_atom in group_atoms:
                  # Check if bond formation would exceed valence
                  mol_curr_valence = mol_atom.GetTotalValence()
                  group_curr_valence = group_atom.GetTotalValence()

                  mol_max_valence = self._get_max_valence(mol_atom.GetAtomicNum())
                  group_max_valence = self._get_max_valence(group_atom.GetAtomicNum())

                  if (mol_curr_valence + 1 <= mol_max_valence and
                      group_curr_valence + 1 <= group_max_valence):

                      try:
                          # Combine molecules
                          combo = Chem.CombineMols(mol_edit, group_edit)
                          editable = Chem.RWMol(combo)

                          # Update property cache
                          for atom in editable.GetAtoms():
                              atom.UpdatePropertyCache(strict=False)

                          # Add bond
                          editable.AddBond(mol_idx,
                                      group_idx + mol_edit.GetNumAtoms(),
                                      Chem.BondType.SINGLE)

                          # Verify molecule
                          try:
                              Chem.SanitizeMol(editable)
                              return editable
                          except:
                              continue
                      except:
                          continue

          return mol

      except Exception as e:
          logging.error(f"Failed to add group: {str(e)}")
          return mol

    def _get_max_valence(self, atomic_num: int) -> int:
        """Get maximum allowed valence for an atom"""
        valence_map = {
            1: 1,   # H
            6: 4,   # C
            7: 3,   # N
            8: 2,   # O
            9: 1,   # F
            15: 5,  # P
            16: 6,  # S
            17: 1   # Cl
        }
        return valence_map.get(atomic_num, 4)  # Default to 4 if unknown

    def _modify_ionizable_groups_safely(self, mol: Chem.RWMol) -> Optional[Chem.RWMol]:
        """Modify ionizable groups with strict valence checking"""
        try:
            patterns = [
                'CN',
                'CO',
                'C(=O)O',
                'CS'
            ]

            for pattern in patterns:
                patt = Chem.MolFromSmarts(pattern)
                if patt:
                    matches = mol.GetSubstructMatches(patt)
                    if matches:
                        match = random.choice(matches)
                        atom_idx = match[0]

                        # Get current atom
                        current_atom = mol.GetAtomWithIdx(atom_idx)
                        current_atom.UpdatePropertyCache(strict=False)

                        # Check valence before modification
                        if pattern == 'CN':
                            if current_atom.GetTotalValence() < 3:  # N max valence
                                new_atom = Chem.Atom(7)  # N
                                new_atom.SetFormalCharge(0)
                                mol.ReplaceAtom(atom_idx, new_atom)
                        elif pattern == 'CO':
                            if current_atom.GetTotalValence() < 2:  # O max valence
                                new_atom = Chem.Atom(8)  # O
                                new_atom.SetFormalCharge(0)
                                mol.ReplaceAtom(atom_idx, new_atom)

            return mol
        except:
            return mol

    def _add_cancer_targeting_groups_safely(self, mol: Chem.RWMol) -> Optional[Chem.RWMol]:
        """Add cancer-targeting groups with valence checking"""
        cancer_groups = [
            'c1ccc(N)cc1',        # Simple aniline (safer)
            'C(=O)NCc1ccccc1',    # Simple benzyl amide (safer)
            'CCN(CC)CC',          # Tertiary amine (safer)
            'CC(=O)N',            # Simple amide (safer)
            'c1cccnc1'            # Pyridine (safer)
        ]
        return self._add_group_with_valence_check(mol, cancer_groups)

    def _modify_molecule_safely(self, mol: Chem.Mol) -> Optional[Chem.Mol]:
        """Safe molecule modification with valence checking"""
        try:
            editable_mol = Chem.RWMol(mol)

            # Update property cache
            for atom in editable_mol.GetAtoms():
                atom.UpdatePropertyCache(strict=False)

            modifications = [
                self._add_cancer_targeting_groups_safely,
                self._modify_ionizable_groups_safely
            ]

            n_modifications = np.random.randint(1, 3)
            for _ in range(n_modifications):
                modification = random.choice(modifications)
                new_mol = modification(editable_mol)
                if new_mol is not None:
                    editable_mol = new_mol

                # Verify valence after each modification
                try:
                    Chem.SanitizeMol(editable_mol)
                except:
                    return mol

            return editable_mol.GetMol()

        except Exception as e:
            logging.error(f"Molecule modification failed: {str(e)}")
            return None

    def _fingerprint_to_mol(self, fp: np.ndarray, similarity_threshold: float = 0.7) -> Optional[Chem.Mol]:
        """Convert fingerprint to molecule with valence safety checks"""
        try:
            if self.dataset is None:
                logging.error("Dataset not initialized")
                return None

            batch_size = 1000
            max_similarity = 0
            best_mol = None

            features = self.dataset.features
            smiles = self.dataset.smiles

            for i in range(0, len(features), batch_size):
                batch_features = features[i:i + batch_size]
                batch_smiles = smiles[i:i + batch_size]
                similarities = np.dot(batch_features, fp)

                batch_max_idx = np.argmax(similarities)
                if similarities[batch_max_idx] > max_similarity:
                    max_similarity = similarities[batch_max_idx]
                    mol = Chem.MolFromSmiles(batch_smiles[batch_max_idx])
                    if mol:
                        # Verify molecule is valid before selecting it
                        try:
                            Chem.SanitizeMol(mol)
                            best_mol = mol
                        except:
                            continue

            if best_mol and max_similarity >= similarity_threshold:
                return self._modify_molecule_safely(best_mol)
            return None

        except Exception as e:
            logging.error(f"Fingerprint conversion failed: {str(e)}")
            return None

def main():
    """Main execution with memory optimization"""
    try:
        # Setup logging
        logging.basicConfig(level=logging.INFO)

        print("Initializing Electroceutical Drug Generator...")
        generator = OptimizedElectroceuticalGenerator()

        # Set up paths
        data_dir = Path("data")
        data_dir.mkdir(exist_ok=True)
        dataset_path = data_dir / "250k_rndm_zinc_drugs_clean_3.csv"

        # Download dataset if needed
        if not dataset_path.exists():
            url = "https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv"
            print(f"Downloading dataset from {url}")
            response = requests.get(url)
            response.raise_for_status()
            dataset_path.write_bytes(response.content)

        # Train model
        print("\nTraining model...")
        generator.train(str(dataset_path), epochs=100, batch_size=32)

        # Generate molecules for different targets
        # targets = [
        #     ('ion_channels', 'potassium'),
        #     ('ion_channels', 'sodium'),
        #     ('membrane_potential', 'hyperpolarization'),
        #     ('bioelectric_signaling', 'gap_junction')
        # ]
        targets = [
            ('ion_channels', 'cancer_ion_regulation')
        ]

        print("\nGenerating molecules...")
        results_dir = Path("results")
        results_dir.mkdir(exist_ok=True)

        for target_type, target_name in targets:
            print(f"\nGenerating for {target_type} - {target_name}")
            molecules = generator.generate_molecules(
                n_samples=10,
                target_type=target_type,
                target_name=target_name,
                min_score=0.6
            )

            # Process and save results
            target_dir = results_dir / f"{target_type}_{target_name}"
            target_dir.mkdir(exist_ok=True)

            # Save summary file
            with open(target_dir / "summary.txt", "w") as f:
                for i, (mol, eval_results) in enumerate(molecules, 1):
                    smiles = Chem.MolToSmiles(mol)
                    f.write(f"Molecule {i}:\n")
                    f.write(f"SMILES: {smiles}\n")
                    f.write(f"Total Score: {eval_results['total_score']:.3f}\n")
                    f.write(f"Bioelectric Score: {eval_results['bioelectric_score']:.3f}\n")
                    f.write(f"Specificity Score: {eval_results['specificity_score']:.3f}\n")
                    f.write(f"Safety Score: {eval_results['safety_score']:.3f}\n")
                    f.write("\nProperties:\n")
                    for prop, value in eval_results['properties'].items():
                        f.write(f"{prop}: {value:.2f}\n")
                    f.write("\n" + "="*50 + "\n")

                    # Save molecule image
                    img = Draw.MolToImage(mol)
                    img.save(target_dir / f"molecule_{i}.png")
                    del img

            # Clean up after each target
            torch.cuda.empty_cache()
            gc.collect()

        print("\nGeneration complete! Results saved in the 'results' directory.")

    except Exception as e:
        logging.error(f"Execution failed: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Initializing Electroceutical Drug Generator...

Training model...
Loading initial data cache...


Streaming output truncated to the last 5000 lines.
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:51:01] DEPRECATION WARNING: please use MorganGenerator
[13:5

Loaded 10000 molecules into cache
Epoch: 5/100, Loss: 145.9947
Epoch: 10/100, Loss: 138.5973
Epoch: 15/100, Loss: 133.6885
Epoch: 20/100, Loss: 129.3581
Epoch: 25/100, Loss: 125.0401
Epoch: 30/100, Loss: 121.5422
Epoch: 35/100, Loss: 118.9750
Epoch: 40/100, Loss: 116.7902
Epoch: 45/100, Loss: 115.1535
Epoch: 50/100, Loss: 113.8053
Epoch: 55/100, Loss: 112.7139
Epoch: 60/100, Loss: 111.5330
Epoch: 65/100, Loss: 110.9148
Epoch: 70/100, Loss: 110.0482
Epoch: 75/100, Loss: 109.6645
Epoch: 80/100, Loss: 109.0897
Epoch: 85/100, Loss: 108.3901
Epoch: 90/100, Loss: 107.9718
Epoch: 95/100, Loss: 107.5011
Epoch: 100/100, Loss: 107.2291

Generating molecules...

Generating for ion_channels - cancer_ion_regulation

Starting molecule generation...
Target: ion_channels/cancer_ion_regulation
Minimum score threshold: 0.6

Attempt 1/1000

Processing fingerprint 1/5
Applying cancer-specific modifications...
Valid molecule generated: CCOC(=O)C[N+](C)(C)C.CC[CH]C(=O)c1ccc2c(c1)OCO2.CN(CC(=O)[O-])C(=O)NCCC

[13:59:02] Can't kekulize mol.  Unkekulized atoms: 11 12 13 14 15 16 23 24 25
[13:59:02] Explicit valence for atom # 7 O, 2, is greater than permitted



Attempt 3/1000

Processing fingerprint 1/5
Applying cancer-specific modifications...
Valid molecule generated: C[C@@H](NC(=O)CN1CCN(c2ccc(C(F)(F)F)cn2)CC1)c1ccc2ccccc2c1

Evaluation results:
Total score: 0.438
Bioelectric score: 0.100
Specificity score: 0.651
Activity score: 0.000
Safety score: 1.000

Molecular properties:
MW: 442.49
LogP: 4.25
TPSA: 48.47
HBA: 4.00
HBD: 1.00
RotatableBonds: 5.00
QED: 0.64

Molecule rejected - score 0.438 below threshold 0.6

Processing fingerprint 2/5
Applying cancer-specific modifications...
Valid molecule generated: Cc1ccccc1CNC(=O)[C@@H]1CCCN(S(=O)(=O)Cc2ccc(Cl)cc2)C1

Evaluation results:
Total score: 0.487
Bioelectric score: 0.100
Specificity score: 0.850
Activity score: 0.000
Safety score: 1.000

Molecular properties:
MW: 420.96
LogP: 3.51
TPSA: 66.48
HBA: 3.00
HBD: 1.00
RotatableBonds: 6.00
QED: 0.78

Molecule rejected - score 0.487 below threshold 0.6

Processing fingerprint 3/5
Applying cancer-specific modifications...
Valid molecule generate

...

Applying cancer-specific modifications...
Valid molecule generated: CCC[C@@H](CC)CCCN1CCN(C)CC1.CCOc1ccc(NC(=O)CNC=O)cc1

Evaluation results:
Total score: 0.442
Bioelectric score: 0.200
Specificity score: 0.567
Activity score: 0.000
Safety score: 1.000

Molecular properties:
MW: 448.65
LogP: 3.61
TPSA: 73.91
HBA: 5.00
HBD: 2.00
RotatableBonds: 13.00
QED: 0.45

Molecule rejected - score 0.442 below threshold 0.6

Attempt 317/1000

Processing fingerprint 1/5
Applying cancer-specific modifications...
Valid molecule generated: C.CCCC.C[N+](C)(C)CC(=O)O.Cc1cc(F)ccc1/C=C\SC(=S)N(C)CC(=O)[O-].O
Invalid valence for atom N at position 19
Total valence: 4
Max allowed: 3
Skipping molecule due to valence issues

Processing fingerprint 2/5
Applying cancer-specific modifications...
Valid molecule generated: Cc1ccccc1NC(=O)Cn1nc2c(-c3nc(-c4cccc(F)c4)no3)cccn2c1=O

Evaluation results:
Total score: 0.512
Bioelectric score: 0.200
Specificity score: 0.850
Activity score: 0.000
Safety score: 1.000

Molecu

[14:00:43] Explicit valence for atom # 32 O, 2, is greater than permitted
[14:00:43] Explicit valence for atom # 23 O, 2, is greater than permitted
[14:00:43] Explicit valence for atom # 29 C, 5, is greater than permitted



Attempt 318/1000

Processing fingerprint 1/5
Applying cancer-specific modifications...
Valid molecule generated: COC(=O)C[N+](C)(C)C.Cc1ccc(C(=O)N2CCN(Cc3ccc(F)cc3)CC2)cc1
Invalid valence for atom N at position 28
Total valence: 4
Max allowed: 3
Skipping molecule due to valence issues

Processing fingerprint 2/5
Applying cancer-specific modifications...
Valid molecule generated: CC[CH][NH+]1CCCC1.CN(CC(=O)[O-])C(=O)CCNc1ccccc1[N+](=O)[O-]
Invalid valence for atom N at position 10
Total valence: 4
Max allowed: 3
Skipping molecule due to valence issues

Processing fingerprint 3/5
Applying cancer-specific modifications...
Valid molecule generated: CC(C)CCN(C)CC(=O)[O-].CNC(=O)COc1ccc(C#N)cc1.c1ccsc1

Evaluation results:
Total score: 0.588
Bioelectric score: 0.400
Specificity score: 0.850
Activity score: 0.100
Safety score: 1.000

Molecular properties:
MW: 432.57
LogP: 2.15
TPSA: 105.49
HBA: 7.00
HBD: 1.00
RotatableBonds: 8.00
QED: 0.69

Molecule rejected - score 0.588 below threshold 0.6

[14:00:43] Explicit valence for atom # 31 C, 5, is greater than permitted
[14:00:43] Explicit valence for atom # 13 O, 2, is greater than permitted
[14:00:43] Explicit valence for atom # 13 O, 2, is greater than permitted
[14:00:43] Explicit valence for atom # 24 C, 5, is greater than permitted
[14:00:43] Explicit valence for atom # 36 O, 2, is greater than permitted
[14:00:43] Explicit valence for atom # 39 O, 2, is greater than permitted



Evaluation results:
Total score: 0.562
Bioelectric score: 0.300
Specificity score: 0.850
Activity score: 0.100
Safety score: 1.000

Molecular properties:
MW: 497.61
LogP: 1.46
TPSA: 121.88
HBA: 7.00
HBD: 3.00
RotatableBonds: 6.00
QED: 0.52

Molecule rejected - score 0.562 below threshold 0.6

Attempt 319/1000

Processing fingerprint 1/5
Applying cancer-specific modifications...
Valid molecule generated: C[NH+](C)CCn1cc(NC(=O)NCCc2cccc(F)c2)cn1
Invalid valence for atom N at position 1
Total valence: 4
Max allowed: 3
Skipping molecule due to valence issues

Processing fingerprint 2/5
Applying cancer-specific modifications...
Valid molecule generated: CCOC(=O)/C([O-])=N/c1nnc([C@@H](C)CC(=O)N2CCN(C)CC2)s1.CCc1ccccc1

Evaluation results:
Total score: 0.625
Bioelectric score: 0.450
Specificity score: 0.850
Activity score: 0.200
Safety score: 1.000

Molecular properties:
MW: 474.61
LogP: 2.01
TPSA: 111.05
HBA: 9.00
HBD: 0.00
RotatableBonds: 6.00
QED: 0.36

Molecule accepted ✓

Generation co